In [24]:
import re
import unidecode

import pandas as pd
import numpy as np

import joblib
import json

from nltk.corpus import stopwords
import nltk
from nltk.stem.snowball import SnowballStemmer

In [28]:
def replace_par_regex(tex:str, langue='francais') ->str :
    
    """
    mets les caractères en minuscule
    supprime les caractères non alphabétiques
    """

    # mettre le texte en minuscule
    pe = tex.lower()


    if langue == 'english':
        pe = re.sub('[^a-z_]','_',pe)
    else:
        pe = re.sub('[^a-zàâçéèêëîïôûùüÿñæœ]', '_', pe)
    
    # supprimer les mots de 1 ou deux lettres
    pe = re.sub('_[a-zàâçéèêëîïôûùüÿñæœ]{1,2}_', '_',pe)

    pe = re.sub('[_]{2,}','_',pe)
    pe = re.sub('_',' ',pe)
    
    # supprimer le l qui reste parfois
    pe = re.sub(' l ',' ', pe)

    return pe





def lemm_stem(dataset:str, type='stem'):
    
    """
    lemmization ou stemming d'un dataset
    pour la lemmization, nécessité d'enlever les accents 
    retourne le dataset traité
    """

    french_stopwords = nltk.corpus.stopwords.words('french')
    stemmer = SnowballStemmer(language='french')
   
    st = ""
    var = unidecode.unidecode(dataset)
    tokenize_sentence = nltk.tokenize.word_tokenize(var)
    words_w_stopwords = [i for i in tokenize_sentence if i not in french_stopwords]
    words_stemming = [stemmer.stem(w) for w in words_w_stopwords]
    for j in range(len(words_stemming)):
        st = st + ' ' + words_stemming[j] + ' '
    
    return st



def intersection(dataframe:str) -> str :
    
    with open('./json/wordsBag.json', 'r') as f:
        sac = np.array(list(json.load(f).keys()))


    phra = np.array(dataframe.split())

    phra = np.intersect1d(phra, sac)

    st = ""
    for i in range(phra.shape[0]):
        st = st + " " + phra[i]


    return st


def prediciton(texte:str):
    
    var = replace_par_regex(texte)
    var = lemm_stem(var)
    var = intersection(var)
    vec = joblib.load('./joblib/cvec.joblib')
    clf = joblib.load('./joblib/model.joblib')

    var = pd.DataFrame(vec.transform(var.split()).todense().tolist())
    res = clf.predict(var)
    pred = pd.Series(res)
    rea = pred.value_counts()


    if rea.index[0] == 1:
        if rea.iloc[0] >= rea.iloc[1]:
            note = 'commentaire positif'
        else:
            note = 'commentaire négatif'
    else:
        if rea.iloc[0] < rea.iloc[1]:
            note = 'commentaire positif'
        else:
            note = 'commentaire négatif'

    return note


## Quelques commentaires issus du site allociné, non issus des données utilisées auparavant

In [29]:
str = "Un film que je connais déjà depuis bien longtemps car découvert lors de mes années lycées et pourtant, encore aujourd’hui, ce film est encore culte pour moi, au point qu’il occupe toujours dans mon cœur une place à part. J’aime cette atmosphère d’anticipation totalement oppressante mais en même temps gentiment absurde. J’aime cet humour noir corrosif. J’aime cette gourde de Sam Lowry remarquablement interprété par Jonathan Pryce, et j’aime encore plus cet Harry Tuttle – merveilleux plombier terroriste (tout un concept) f – auquel le génial Robert de Niro donne vie. Alors voilà, Brazil c’est de la création visuelle remarquable, une mise en scène parfaite de Terry Gilliam et aussi une bande originale de Michael Kamen qui reste gravé pour l’éternité dans ma mémoire. Donc voilà, si vous voulez savoir ce qui peut susciter ça chez quelqu’un comme moi, vous savez ce qu’il vous reste à faire…"
str1 = "Décidément, Terry Gilliam, que ce soit pour la science-fiction traditionnelle (L'Armée des 12 singes), ou la science fiction burlesque avec donc ce Brazil, je n'accroche vraiment pas. Et pourtant Dieu sait que j'apprécie particulièrement les films d'anticipation aux histoires dystopiques et tout, mais là c'est vraiment... spécial... très spécial...c'est anglais quoi... ce film n'est pourtant pas dénué de bonne idée avec par exemple ce scénario assez original à la base qui prend avec dérision et ironie les travers de notre société, ou même avec ce pitch scénaristique en lui même où un bug informatique dérègle toute cette societé, mais Gilliam n'exploite pas bien ça en profonfeur et surtout, semble ici ne jamais trouver son juste milieu et se perd dans ce qui peut s'apparenter à un melting-pot des genres assez fouillis et vain dans sa globalité. comme cela a déjà été dit ça oscille entre de la sf, du drame, de la comédie, de la romance ... etc et aucun de ces genres n'est bien dévelloppé au final, ça tend donc vers un exercice des styles frisant l'excès de style justement... Si bien que l'on finit par s'emmêler les pinceaux d'un point de vu narratif. Je voulais dire aussi que l'intrigue s'essouffle à partir du second tiers notamment, mais encore faudrait-il que ça se lance véritablement... c'est bourré de longueur, longuet (d'où je fais un pléonasme ?), la mise en scène et la réflexion se veulent poétiques, parfois métaphoriques et fantasmagoriques, mais tout ça ne prend jamais, et l'ensemble est tellement décousu et pas drôle pour moi (je ne suis vraiment pas sensible à l'humour Monty Pithon, que ce soit dans les dialogues ou bien avec les gags en eux mêmes ) donc on s'ennuie ferme. Mention spéciale au rêve à la con récurrent de ce fontionnaire benêt : d'un chiant abyssal tellement ça n'apporte rien au récit... en plus de ça le héro a le charisme d'un bloc de parpaing, donc pour être intéressé par ces états-d'âme ou bien son sort... Et comme je ne suis pas sensible non plus à l'artistique global du film à part un pan d'architecture par ci par là et quelques clins d'oeil à des classiques de sf, ça fait que j'ai du me forcer pour resté jusqu'au générique final avec ce dénouement qui joue la carte de l'imaginaire si j'ai bien compris. Je ne lui trouve que peu d'intérêt en définitif à part peut-être la bande-sonore vraiment bien dans le ton du film (même si là encore l'excès de certaines autres musiques finit par nous fatiguer) ainsi qu'une bonne petite galerie de personnage bien loufoque comme avec la mère de ce protagoniste principal ou bien évidemment ce second rôle campé par le peu présent De Niro. Pour finir sur l'interprétation globale là je n'aurais en revanche pas grand chose à reprocher à ce Brazil , c'est parfaitement dans le ton du film, c'est ce à quoi je m'attendais et ça le mérite de ne pas trop faire dans le cabotinage je trouve; à part avec la VF peut-être mais sinon c'est relativement bien dosée de ce côté là. Nouvelle déception donc pour ce nouveau Gilliam que je me faisais une joie de découvrir à l'avance."
str2 = 'Ce film vieillit mal mais il reste sympa à visionner pour la beauté naturelle et la grâce inégalable de Rita Hayworth. Une romance musicale en noir et blanc pour les amateurs du genre.'
str3 = 'Bonne interprétation et bonne idée et si le début est plaisant la suite devient vite ennuyeuse, la mise en scène manque de punch pour nous accrocher.'
# 1 0 0 0 pour rappel

## Prédiction

Rentrez une chaîne de caractères dans la fonction prédiction

In [30]:
prediciton(str3)

'commentaire négatif'